In [1]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import joblib
from sklearn.tree import DecisionTreeRegressor

# Inisialisasi Firebase Admin SDK
cred = credentials.Certificate('kebuncabe-6f515-firebase-adminsdk-n54zr-c89d1f51dd.json')

try:
    firebase_admin.get_app()
except ValueError:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://kebuncabe-6f515-default-rtdb.firebaseio.com/'
    })

# Memuat model
model = joblib.load('C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\fertilizer_and_water_model.pkl')

# Fungsi untuk prediksi secara realtime
def predict_fertilizer_realtime():
    # Baca data dari Firebase Realtime Database
    ref = db.reference('sensorData')
    sensor_data = ref.get()

    if not sensor_data:
        print('Tidak ada data sensor di Firebase.')
        return

    # List untuk menyimpan data sensor yang valid
    sensor_records = []

    # Iterasi dan validasi data sensor
    for key, value in sensor_data.items():
        if isinstance(value, dict) and all(k in value for k in ['moisture', 'ph', 'N', 'P', 'K']):
            sensor_records.append(value)
        else:
            print(f"Data tidak valid untuk kunci {key}. Data diabaikan.")

    # Jika tidak ada data yang valid, keluar dari fungsi
    if not sensor_records:
        print('Tidak ada data sensor yang valid untuk diproses.')
        return

    # Buat DataFrame dari list data sensor yang valid
    sensor_df = pd.DataFrame(sensor_records)

    # Lakukan prediksi jumlah pupuk yang dibutuhkan dan kebutuhan air
    predicted_values = model.predict(sensor_df[['moisture', 'ph', 'N', 'P', 'K']])
    predicted_fertilizers_grams = predicted_values[:, :3] * 0.5  # Konversi hanya kolom pupuk ke gram
    predicted_water_ml = predicted_values[:, 3]  # Kolom air tetap dalam ml

    # Tambahkan hasil prediksi ke DataFrame sensor_df
    sensor_df['Penambahan Pupuk Urea (N)'] = predicted_fertilizers_grams[:, 0]
    sensor_df['Penambahan Pupuk SP36 (P)'] = predicted_fertilizers_grams[:, 1]
    sensor_df['Penambahan Pupuk KCL (K)'] = predicted_fertilizers_grams[:, 2]
    sensor_df['Penambahan Air (ml)'] = predicted_water_ml

    # Simpan prediksi ke Firebase Realtime Database
    predictions = sensor_df[[
        'Penambahan Pupuk Urea (N)', 
        'Penambahan Pupuk SP36 (P)', 
        'Penambahan Pupuk KCL (K)', 
        'Penambahan Air (ml)'
    ]].to_dict(orient='records')

    predictions_ref = db.reference('predictions_realtime')
    for i, prediction in enumerate(predictions):
        predictions_ref.child(str(i)).set(prediction)

    print('Prediksi untuk data realtime telah disimpan ke Firebase.')

# Panggil fungsi untuk melakukan prediksi secara realtime
predict_fertilizer_realtime()

Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.


In [12]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import joblib
from sklearn.tree import DecisionTreeRegressor

# Inisialisasi Firebase Admin SDK
cred = credentials.Certificate('kebuncabe-6f515-firebase-adminsdk-n54zr-c89d1f51dd.json')

try:
    firebase_admin.get_app()
except ValueError:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://kebuncabe-6f515-default-rtdb.firebaseio.com/'
    })

# Memuat model
model = joblib.load('C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\fertilizer_and_water_model.pkl')

# Fungsi untuk prediksi secara realtime
def predict_fertilizer_realtime():
    # Baca data dari Firebase Realtime Database
    ref = db.reference('sensorData')
    sensor_data = ref.get()

    if not sensor_data:
        print('Tidak ada data sensor di Firebase.')
        return

    # List untuk menyimpan data sensor yang valid
    sensor_records = []

    # Iterasi dan validasi data sensor
    for key, value in sensor_data.items():
        if isinstance(value, dict) and all(k in value for k in ['moisture', 'ph', 'N', 'P', 'K']):
            sensor_records.append(value)
        else:
            print(f"Data tidak valid untuk kunci {key}. Data diabaikan.")

    # Jika tidak ada data yang valid, keluar dari fungsi
    if not sensor_records:
        print('Tidak ada data sensor yang valid untuk diproses.')
        return

    # Buat DataFrame dari list data sensor yang valid
    sensor_df = pd.DataFrame(sensor_records)

    # Lakukan prediksi jumlah pupuk yang dibutuhkan dan kebutuhan air
    predicted_values = model.predict(sensor_df[['moisture', 'ph', 'N', 'P', 'K']])
    print("Predicted values:", predicted_values)  # Debugging output

    predicted_fertilizers_grams = predicted_values[:, :3] * 0.5  # Konversi hanya kolom pupuk ke gram
    predicted_water_ml = predicted_values[:, 3]  # Kolom air tetap dalam ml

    # Tambahkan hasil prediksi ke DataFrame sensor_df
    sensor_df['Penambahan Pupuk Urea (N)'] = predicted_fertilizers_grams[:, 0]
    sensor_df['Penambahan Pupuk SP36 (P)'] = predicted_fertilizers_grams[:, 1]
    sensor_df['Penambahan Pupuk KCL (K)'] = predicted_fertilizers_grams[:, 2]
    sensor_df['Penambahan Air (ml)'] = predicted_water_ml

    # Output untuk debugging
    print("DataFrame dengan prediksi:")
    print(sensor_df[['Penambahan Pupuk Urea (N)', 'Penambahan Pupuk SP36 (P)', 'Penambahan Pupuk KCL (K)', 'Penambahan Air (ml)']])

    # Simpan prediksi ke Firebase Realtime Database
    predictions = sensor_df[[
        'Penambahan Pupuk Urea (N)', 
        'Penambahan Pupuk SP36 (P)', 
        'Penambahan Pupuk KCL (K)', 
        'Penambahan Air (ml)'
    ]].to_dict(orient='records')

    predictions_ref = db.reference('predictions_realtime')
    for i, prediction in enumerate(predictions):
        predictions_ref.child(str(i)).set(prediction)

    print('Prediksi untuk data realtime telah disimpan ke Firebase.')

# Panggil fungsi untuk melakukan prediksi secara realtime
predict_fertilizer_realtime()


Data tidak valid untuk kunci waterLevel. Data diabaikan.
Predicted values: [[0. 0. 0. 0.]]
DataFrame dengan prediksi:
   Penambahan Pupuk Urea (N)  Penambahan Pupuk SP36 (P)  \
0                        0.0                        0.0   

   Penambahan Pupuk KCL (K)  Penambahan Air (ml)  
0                       0.0                  0.0  
Prediksi untuk data realtime telah disimpan ke Firebase.


In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import joblib
import time

# Inisialisasi Firebase Admin SDK
cred = credentials.Certificate('kebuncabe-6f515-firebase-adminsdk-n54zr-c89d1f51dd.json')

try:
    firebase_admin.get_app()
except ValueError:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://kebuncabe-6f515-default-rtdb.firebaseio.com/'
    })

# Memuat model
model = joblib.load('C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\fertilizer_and_water_model.pkl')

# Fungsi untuk prediksi secara realtime
def predict_fertilizer_realtime():
    while True:
        # Baca data dari Firebase Realtime Database
        ref = db.reference('sensorData')
        sensor_data = ref.get()

        if not sensor_data:
            print('Tidak ada data sensor di Firebase.')
            time.sleep(60)
            continue

        # List untuk menyimpan data sensor yang valid
        sensor_records = []

        # Iterasi dan validasi data sensor
        for key, value in sensor_data.items():
            if isinstance(value, dict) and all(k in value for k in ['moisture', 'ph', 'N', 'P', 'K']):
                sensor_records.append(value)
            else:
                print(f"Data tidak valid untuk kunci {key}. Data diabaikan.")

        # Jika tidak ada data yang valid, keluar dari fungsi
        if not sensor_records:
            print('Tidak ada data sensor yang valid untuk diproses.')
            time.sleep(60)
            continue

        # Buat DataFrame dari list data sensor yang valid
        sensor_df = pd.DataFrame(sensor_records)

        # Lakukan prediksi jumlah pupuk yang dibutuhkan dan kebutuhan air
        predicted_values = model.predict(sensor_df[['moisture', 'ph', 'N', 'P', 'K']])
        predicted_fertilizers_grams = predicted_values[:, :3] * 0.5  # Konversi hanya kolom pupuk ke gram
        predicted_water_ml = predicted_values[:, 3]  # Kolom air tetap dalam ml

        # Tambahkan hasil prediksi ke DataFrame sensor_df
        sensor_df['Penambahan Pupuk Urea (N)'] = predicted_fertilizers_grams[:, 0]
        sensor_df['Penambahan Pupuk SP36 (P)'] = predicted_fertilizers_grams[:, 1]
        sensor_df['Penambahan Pupuk KCL (K)'] = predicted_fertilizers_grams[:, 2]
        sensor_df['Penambahan Air (ml)'] = predicted_water_ml

        # Simpan prediksi ke Firebase Realtime Database
        predictions = sensor_df[[
            'Penambahan Pupuk Urea (N)', 
            'Penambahan Pupuk SP36 (P)', 
            'Penambahan Pupuk KCL (K)', 
            'Penambahan Air (ml)'
        ]].to_dict(orient='records')

        predictions_ref = db.reference('predictions_realtime')
        for i, prediction in enumerate(predictions):
            predictions_ref.child(str(i)).set(prediction)

        print('Prediksi untuk data realtime telah disimpan ke Firebase.')
        
        # Delay sebelum pembacaan berikutnya
        time.sleep(60)

# Panggil fungsi untuk melakukan prediksi secara realtime
predict_fertilizer_realtime()


Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime te

In [2]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import joblib

# Inisialisasi Firebase Admin SDK
cred = credentials.Certificate('kebuncabe-6f515-firebase-adminsdk-n54zr-c89d1f51dd.json')

try:
    firebase_admin.get_app()
except ValueError:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://kebuncabe-6f515-default-rtdb.firebaseio.com/'
    })

# Memuat model
model = joblib.load('C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\fertilizer_and_water_model.pkl')

def predict_fertilizer_realtime():
    ref = db.reference('sensorData')
    sensor_data = ref.get()

    if not sensor_data:
        print('Tidak ada data sensor di Firebase.')
        return

    sensor_records = [value for key, value in sensor_data.items() if isinstance(value, dict) and all(k in value for k in ['moisture', 'ph', 'N', 'P', 'K'])]
    if not sensor_records:
        print('Tidak ada data sensor yang valid untuk diproses.')
        return

    sensor_df = pd.DataFrame(sensor_records)
    predicted_values = model.predict(sensor_df[['moisture', 'ph', 'N', 'P', 'K']])
    predicted_fertilizers_grams = predicted_values[:, :3] * 0.5
    predicted_water_ml = predicted_values[:, 3]

    # Menyimpan prediksi ke Firebase
    predictions = sensor_df[['moisture', 'ph', 'N', 'P', 'K']].copy()
    predictions['Penambahan Pupuk Urea (N)'] = predicted_fertilizers_grams[:, 0]
    predictions['Penambahan Pupuk SP36 (P)'] = predicted_fertilizers_grams[:, 1]
    predictions['Penambahan Pupuk KCL (K)'] = predicted_fertilizers_grams[:, 2]
    predictions['Penambahan Air (ml)'] = predicted_water_ml

    predictions_ref = db.reference('predictions_realtime')
    for i, prediction in enumerate(predictions.to_dict(orient='records')):
        predictions_ref.child(str(i)).set(prediction)

    # Mengirim perintah untuk menggerakkan relay
    if predicted_water_ml[0] > 0:
        db.reference('relayControl').set({'relay': 1})  # Nyalakan relay
    else:
        db.reference('relayControl').set({'relay': 0})  # Matikan relay

    # Mengirim perintah untuk menggerakkan servo
    db.reference('servoControl').set({
        'servoUrea': predicted_fertilizers_grams[0, 0],
        'servoSP36': predicted_fertilizers_grams[0, 1],
        'servoKCL': predicted_fertilizers_grams[0, 2]
    })

    print('Prediksi untuk data realtime telah disimpan ke Firebase, relay, dan servo telah diperbarui.')

predict_fertilizer_realtime()


Prediksi untuk data realtime telah disimpan ke Firebase, relay, dan servo telah diperbarui.


In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import joblib
import time

# Inisialisasi Firebase Admin SDK
cred = credentials.Certificate('kebuncabe-6f515-firebase-adminsdk-n54zr-c89d1f51dd.json')

try:
    firebase_admin.get_app()
except ValueError:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://kebuncabe-6f515-default-rtdb.firebaseio.com/'
    })

# Memuat model
model = joblib.load('C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\fertilizer_and_water_model.pkl')

# Fungsi untuk prediksi secara realtime
def predict_fertilizer_realtime():
    while True:
        # Baca data dari Firebase Realtime Database
        ref = db.reference('sensorData')
        sensor_data = ref.get()

        if not sensor_data:
            print('Tidak ada data sensor di Firebase.')
            time.sleep(60)
            continue

        # List untuk menyimpan data sensor yang valid
        sensor_records = []

        # Iterasi dan validasi data sensor
        for key, value in sensor_data.items():
            if isinstance(value, dict) and all(k in value for k in ['moisture', 'ph', 'N', 'P', 'K']):
                sensor_records.append(value)
            else:
                print(f"Data tidak valid untuk kunci {key}. Data diabaikan.")

        # Jika tidak ada data yang valid, keluar dari fungsi
        if not sensor_records:
            print('Tidak ada data sensor yang valid untuk diproses.')
            time.sleep(60)
            continue

        # Buat DataFrame dari list data sensor yang valid
        sensor_df = pd.DataFrame(sensor_records)

        # Lakukan prediksi jumlah pupuk yang dibutuhkan dan kebutuhan air
        predicted_values = model.predict(sensor_df[['moisture', 'ph', 'N', 'P', 'K']])

        # Konversi hasil prediksi dari ppm ke gram
        predicted_fertilizers_grams = predicted_values[:, :3] * 0.5  # Konversi hanya kolom pupuk ke gram
        predicted_water_ml = predicted_values[:, 3]  # Kolom air tetap dalam ml

        # Tambahkan hasil prediksi ke DataFrame sensor_df
        sensor_df['Penambahan Pupuk Urea (N)'] = predicted_fertilizers_grams[:, 0]
        sensor_df['Penambahan Pupuk SP36 (P)'] = predicted_fertilizers_grams[:, 1]
        sensor_df['Penambahan Pupuk KCL (K)'] = predicted_fertilizers_grams[:, 2]
        sensor_df['Penambahan Air (ml)'] = predicted_water_ml

        # Simpan prediksi ke Firebase Realtime Database
        predictions = sensor_df[[
            'Penambahan Pupuk Urea (N)', 
            'Penambahan Pupuk SP36 (P)', 
            'Penambahan Pupuk KCL (K)', 
            'Penambahan Air (ml)'
        ]].to_dict(orient='records')

        predictions_ref = db.reference('predictions_realtime')
        for i, prediction in enumerate(predictions):
            predictions_ref.child(str(i)).set(prediction)

        print('Prediksi untuk data realtime telah disimpan ke Firebase.')
        
        # Delay sebelum pembacaan berikutnya
        time.sleep(60)

# Panggil fungsi untuk melakukan prediksi secara realtime
predict_fertilizer_realtime()


Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime telah disimpan ke Firebase.
Data tidak valid untuk kunci waterLevel. Data diabaikan.
Prediksi untuk data realtime te

In [ ]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Baca dataset
data_path = 'C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\dataujibetul.csv'
dataset_with_fertilizer_path = 'C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\dataset_with_fertilizer_and_water.csv'
predicted_data_path = 'C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\predicted_fertilizer_and_water.csv'
model_path = 'C:\\Users\\ASUS\\Documents\\TUGAS AKHIRKU\\fertilizer_and_water_model.pkl'

# Baca dataset awal
data = pd.read_csv(data_path)

# Konversi kolom ke numerik
data['N'] = pd.to_numeric(data['N'], errors='coerce')
data['P'] = pd.to_numeric(data['P'], errors='coerce')
data['K'] = pd.to_numeric(data['K'], errors='coerce')
data['moisture'] = pd.to_numeric(data['moisture'], errors='coerce')

# Batas tiap n, p, k, dan kelembaban
limits = {
    'N': {'tooLittle': 50, 'idealMin': 50, 'idealMax': 199, 'tooMuch': 200},
    'P': {'tooLittle': 4, 'idealMin': 4, 'idealMax': 13, 'tooMuch': 14},
    'K': {'tooLittle': 50, 'idealMin': 50, 'idealMax': 199, 'tooMuch': 200},
    'moisture': {'tooLittle': 50, 'idealMin': 50, 'idealMax': 80, 'tooMuch': 80}
}

# Fungsi untuk menghitung kebutuhan pupuk
def calculate_fertilizer(n, p, k):
    N_fertilizer = 0
    P_fertilizer = 0
    K_fertilizer = 0
    
    if n < limits['N']['tooLittle']:
        N_fertilizer = limits['N']['idealMin'] - n
    if p < limits['P']['tooLittle']:
        P_fertilizer = limits['P']['idealMin'] - p
    if k < limits['K']['tooLittle']:
        K_fertilizer = limits['K']['idealMin'] - k

    return N_fertilizer, P_fertilizer, K_fertilizer

# Fungsi untuk menghitung kebutuhan air
def calculate_water(moisture):
    water_needed = 0
    
    if moisture < limits['moisture']['tooLittle']:
        water_needed = limits['moisture']['idealMin'] - moisture

    water_needed_ml = water_needed * 10
    return water_needed_ml

# Kolom tambahan untuk masing-masing N, P, K, dan kelembaban
data['Urea (N)'] = data.apply(lambda row: calculate_fertilizer(row['N'], row['P'], row['K'])[0], axis=1)
data['SP36 (P)'] = data.apply(lambda row: calculate_fertilizer(row['N'], row['P'], row['K'])[1], axis=1)
data['KCL (K)'] = data.apply(lambda row: calculate_fertilizer(row['N'], row['P'], row['K'])[2], axis=1)
data['Air (ml)'] = data.apply(lambda row: calculate_water(row['moisture']), axis=1)

# Menyimpan dataset baru
data.to_csv(dataset_with_fertilizer_path, index=False)
print(f'Dataset dengan nilai pupuk dan air telah disimpan ke {dataset_with_fertilizer_path}')

# Baca dataset dengan kolom tambahan
dataset = pd.read_csv(dataset_with_fertilizer_path)

# Fitur dan label
X = dataset[['moisture', 'ph', 'N', 'P', 'K']]
y = dataset[['Urea (N)', 'SP36 (P)', 'KCL (K)', 'Air (ml)']]

# Pembagian dataset untuk training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat model decision tree
model = DecisionTreeRegressor()

# Latih model
model.fit(X_train, y_train)

# Simpan model
joblib.dump(model, model_path)
print(f'Model telah disimpan ke {model_path}')

# Prediksi dengan data uji
y_pred = model.predict(X_test)

# Hitung metrik evaluasi
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R²): {r2}')

# Baca dataset baru
new_data = pd.read_csv(dataset_with_fertilizer_path)

# Prediksi jumlah pupuk dan air yang diperlukan
predicted_values = model.predict(new_data[['moisture', 'ph', 'N', 'P', 'K']])

# Konversi hasil prediksi ke satuan gram untuk pupuk dan ml untuk air
predicted_fertilizers_grams = predicted_values[:, :3] * 0.5
predicted_water_ml = predicted_values[:, 3]

# Tambah prediksi ke data baru
new_data['Urea (N)'] = predicted_fertilizers_grams[:, 0]
new_data['SP36 (P)'] = predicted_fertilizers_grams[:, 1]
new_data['KCL (K)'] = predicted_fertilizers_grams[:, 2]
new_data['Air (ml)'] = predicted_water_ml

# Simpan data baru dengan prediksi
new_data.to_csv(predicted_data_path, index=False)
print(f'Prediksi telah disimpan ke {predicted_data_path}')

# Tampilkan hasil prediksi
for index, row in new_data.iterrows():
    print(f"Prediksi penambahan pupuk dan air untuk data ke-{index + 1}:")
    print(f"Moisture: {row['moisture']}, pH: {row['ph']}, N: {row['N']}, P: {row['P']}, K: {row['K']}")
    print(f'Urea (N): {row["Urea (N)"]:.2f} gram')
    print(f'SP36 (P): {row["SP36 (P)"]:.2f} gram')
    print(f'KCL (K): {row["KCL (K)"]:.2f} gram')
    print(f'Air: {row["Air (ml)"]:.2f} ml')
    print('---------------------')
